In [131]:
import numpy as np
import pandas as pd

from random import random
from scipy.optimize import linprog

# Условия

$A > 0$

$size(A) = (8, 6)$

$b, c > 0$

---

$c \cdot x \to \max;\ Ax \le b, x \ge 0$

# Решение

## Прямая задача

### Основной код

In [132]:
def simplex(c, A, b):
    # Таблица
    n, m = A.shape
    table = np.zeros((n, m + n), dtype=float)

    # Ограничения
    for i in range(m):
        for j in range(n):
            table[j, i] = A[j, i]
    # Базисные переменные
    for i in range(m, n + m):
        for j in range(n):
            if i - m == j:
                table[j, i] = 1
            else:
                table[j, i] = 0

    b = b

    # Задание коэффициентов функции
    c_old = c.copy()

    c = np.zeros(n + m, dtype=float)
    for i in range(m):
        c[i] = c_old[i]

    Cb = np.zeros(n, dtype=float)
    """Вектор коэффициентов базисных переменных"""
    Bp = np.array([i for i in range(m, n + m)], dtype=float)  
    """Индексы текущих базисных переменных"""
    result = np.zeros(n + m)
    """"""

    while True:
        F = sum(Cb * b)

        simplex_matrix = np.zeros((n, 4 + m + n), dtype=float)
        simplex_matrix[:, 0:1] = Bp.reshape(n, 1)
        simplex_matrix[:, 1:2] = Cb.reshape(n, 1)
        simplex_matrix[:, 2:3] = b.reshape(n, 1)
        simplex_matrix[:, 3:-1] = table

        deltas = np.array([c[i] - sum(Cb[j] * table[j, i] for j in range(n)) for i in range(n + m)], dtype=float)
        
        print(f'Промежуточный F_max = {np.round(F, 2)}')

        k = np.argmax(deltas)

        fractions = np.array([b[i] / table[i, k] for i in range(n)], dtype=float)
        fractions = np.array([fractions[i] if fractions[i] > 0 else np.inf for i in range(n)], dtype=float)

        simplex_matrix[:, n + m + 3:n + m + 4] = fractions.reshape(n, 1)

        print(f'Δ = {np.round(deltas, 2)}')

        flag = True
        for i in range(n + m):
            if deltas[i] > 0:
                flag = False
                break
        
        if flag:
            return np.round(F, 2), np.round(result, 2)

        print(f'simpex table: \n{pd.DataFrame(np.round(simplex_matrix, 2))}')

        r = np.argmin(fractions)

        result[k] = fractions[r] * deltas[k] / c[k]

        table_old = table.copy()

        for i in range(n + m):
            for j in range(n):
                if j == r:
                    table[j, i] = table_old[j, i] / table_old[r, k]
                else:
                    table[j, i] = table_old[j, i] - (table_old[r, i] * table_old[j, k] / table_old[r, k])

        Bp[r] = k
        Cb[r] = c[k]

        b_old = b.copy()
        for i in range(n + m):
            for j in range(n):
                if j == r:
                    b[j] = b_old[j] / table_old[r, k]
                else:
                    b[j] = b_old[j] - (b_old[r] * table_old[j, k] / table_old[r, k])

        print('-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-')

### Тестирование

#### Данные с задачи

In [133]:
# Пример использования
A = np.matrix("2 4;"
              "1 1;"
              "2 1", dtype=float)
b = np.array([560, 170, 300], dtype=float)
c = np.array([4, 5], dtype=float)

optimal_solution, optimal_value = simplex(c, A, b)
print("Оптимальное решение:", optimal_solution)
print("Оптимальная цель:", optimal_value)


Промежуточный F_max = 0.0
Δ = [4. 5. 0. 0. 0.]
simpex table: 
     0    1      2    3    4    5    6    7      8
0  2.0  0.0  560.0  2.0  4.0  1.0  0.0  0.0  140.0
1  3.0  0.0  170.0  1.0  1.0  0.0  1.0  0.0  170.0
2  4.0  0.0  300.0  2.0  1.0  0.0  0.0  1.0  300.0
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
Промежуточный F_max = 700.0
Δ = [ 1.5   0.   -1.25  0.    0.  ]
simpex table: 
     0    1      2    3    4     5    6    7       8
0  1.0  5.0  140.0  0.5  1.0  0.25  0.0  0.0  280.00
1  3.0  0.0   30.0  0.5  0.0 -0.25  1.0  0.0   60.00
2  4.0  0.0  160.0  1.5  0.0 -0.25  0.0  1.0  106.67
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
Промежуточный F_max = 790.0
Δ = [ 0.   0.  -0.5 -3.   0. ]
Оптимальное решение: 790.0
Оптимальная цель: [ 22.5 140.    0.    0.    0. ]


C:\Users\artem\AppData\Local\Temp\ipykernel_19400\3525270436.py:49: RuntimeWarning: divide by zero encountered in scalar divide
  fractions = np.array([b[i] / table[i, k] for i in range(n)], dtype=float)


#### Случайные данные

In [134]:
c = np.array([2 + round(2 * random(), 1) for x in range(6)], dtype=float)  
print(F'{c=}\n')

A = np.array([[2 + round(15 * random(), 1) for x in range(6)] for y in range(8)], dtype=float)
print(F'A={pd.DataFrame(A)}\n')

b = np.array([50 + round(50 * random(), 0) for y in range(8)], dtype=float)
print(F'{b=}\n')

c=array([2.3, 2.5, 2.5, 3. , 3.1, 2.8])

A=      0     1     2     3     4     5
0   2.4   7.9   6.7   6.8  10.7   9.5
1  15.4  13.3  14.0   7.3  15.9   4.3
2   8.2  11.3  13.8   9.7  12.5   8.1
3   3.8   7.2   8.2   7.4  10.5   7.5
4   8.2  14.9   8.9   8.4   6.6   4.8
5  12.8  15.1   8.4   8.2   3.5  16.2
6   7.1  13.1  12.0   2.7  16.7  15.0
7  12.5  12.3  12.1  10.1   2.6  10.2

b=array([57., 81., 78., 82., 77., 68., 61., 73.])



In [135]:
F_max, result = simplex(c, A, b)
print(f'F_max = {F_max}\nОптимальные значения переменных: \n{result}')
sum(result[:c.shape[0]] * c)

Промежуточный F_max = 0.0
Δ = [2.3 2.5 2.5 3.  3.1 2.8 0.  0.  0.  0.  0.  0.  0.  0. ]
simpex table: 
     0    1     2     3     4     5     6     7     8    9    10   11   12  \
0   6.0  0.0  57.0   2.4   7.9   6.7   6.8  10.7   9.5  1.0  0.0  0.0  0.0   
1   7.0  0.0  81.0  15.4  13.3  14.0   7.3  15.9   4.3  0.0  1.0  0.0  0.0   
2   8.0  0.0  78.0   8.2  11.3  13.8   9.7  12.5   8.1  0.0  0.0  1.0  0.0   
3   9.0  0.0  82.0   3.8   7.2   8.2   7.4  10.5   7.5  0.0  0.0  0.0  1.0   
4  10.0  0.0  77.0   8.2  14.9   8.9   8.4   6.6   4.8  0.0  0.0  0.0  0.0   
5  11.0  0.0  68.0  12.8  15.1   8.4   8.2   3.5  16.2  0.0  0.0  0.0  0.0   
6  12.0  0.0  61.0   7.1  13.1  12.0   2.7  16.7  15.0  0.0  0.0  0.0  0.0   
7  13.0  0.0  73.0  12.5  12.3  12.1  10.1   2.6  10.2  0.0  0.0  0.0  0.0   

    13   14   15   16     17  
0  0.0  0.0  0.0  0.0   5.33  
1  0.0  0.0  0.0  0.0   5.09  
2  0.0  0.0  0.0  0.0   6.24  
3  0.0  0.0  0.0  0.0   7.81  
4  1.0  0.0  0.0  0.0  11.67  
5  0.0  

C:\Users\artem\AppData\Local\Temp\ipykernel_19400\3525270436.py:69: RuntimeWarning: divide by zero encountered in scalar divide
  result[k] = fractions[r] * deltas[k] / c[k]
C:\Users\artem\AppData\Local\Temp\ipykernel_19400\3525270436.py:69: RuntimeWarning: divide by zero encountered in scalar divide
  result[k] = fractions[r] * deltas[k] / c[k]
C:\Users\artem\AppData\Local\Temp\ipykernel_19400\3525270436.py:69: RuntimeWarning: divide by zero encountered in scalar divide
  result[k] = fractions[r] * deltas[k] / c[k]


simpex table: 
     0    1      2    3     4     5    6    7    8     9    10    11   12  \
0   3.0  3.0   5.36 -0.0  0.00  0.81  1.0  0.0  0.0  0.22  0.0 -0.19  0.0   
1   0.0  2.3   0.73  1.0  0.61  0.60  0.0  0.0  0.0 -0.24  0.0  0.19  0.0   
2   5.0  2.8   0.65  0.0  0.40 -0.44  0.0  0.0  1.0  0.09  0.0 -0.09  0.0   
3   9.0  0.0  22.30 -0.0 -0.70 -0.64  0.0  0.0  0.0 -0.63  0.0 -0.31  1.0   
4  10.0  0.0  15.07 -0.0  6.38 -3.13 -0.0  0.0 -0.0  0.15  0.0 -0.62  0.0   
5   7.0  0.0   9.07 -0.0 -1.68 -5.07  0.0  0.0  0.0  2.83  1.0 -3.75  0.0   
6   4.0  3.1   1.18  0.0  0.25  0.37  0.0  1.0  0.0 -0.07  0.0  0.16  0.0   
7  12.0  0.0  11.89 -0.0 -1.32  6.07  0.0  0.0  0.0  0.95  0.0 -2.12  0.0   

    13    14   15    16      17  
0  0.0 -0.24 -0.0  0.33     inf  
1  0.0  0.12  0.0 -0.13    6.08  
2  0.0  0.08  0.0 -0.03    8.60  
3  0.0  0.14  0.0 -0.12  160.43  
4  1.0  0.28  0.0 -0.57   53.23  
5  0.0 -1.35  0.0  2.07     inf  
6  0.0  0.06  0.0 -0.15   20.09  
7  0.0 -2.32  1.0  

C:\Users\artem\AppData\Local\Temp\ipykernel_19400\3525270436.py:69: RuntimeWarning: divide by zero encountered in scalar divide
  result[k] = fractions[r] * deltas[k] / c[k]
C:\Users\artem\AppData\Local\Temp\ipykernel_19400\3525270436.py:49: RuntimeWarning: divide by zero encountered in scalar divide
  fractions = np.array([b[i] / table[i, k] for i in range(n)], dtype=float)
C:\Users\artem\AppData\Local\Temp\ipykernel_19400\3525270436.py:69: RuntimeWarning: divide by zero encountered in scalar divide
  result[k] = fractions[r] * deltas[k] / c[k]
C:\Users\artem\AppData\Local\Temp\ipykernel_19400\3525270436.py:49: RuntimeWarning: divide by zero encountered in scalar divide
  fractions = np.array([b[i] / table[i, k] for i in range(n)], dtype=float)


22.506999999999998

## Двойственная задача

### Основной код

In [136]:
def dual_simplex(A, b, c, M):
    
    def get_optimal_values(Bp, b, n, m):
        optimal_values = np.zeros(2 * n + m)  # Creates an array to hold the optimal values
        for i in range(len(Bp)):
            if Bp[i] < len(optimal_values):  # Ensure we do not go out of bounds
                optimal_values[int(Bp[i])] = b[i]
        return optimal_values

    n, m = A.shape

    table = np.zeros((n, m + 2 * n), dtype=float)

    for i in range(m):
        for j in range(n):
            table[j, i] = A[j, i]

    for i in range(m, n + m):
        for j in range(n):
            if i - m == j:
                table[j, i] = -1
            else:
                table[j, i] = 0

    for i in range(n + m, 2 * n + m):
        for j in range(n):
            if i - m - n == j:
                table[j, i] = 1
            else:
                table[j, i] = 0

    b = b

    c_old = c.copy()

    c = np.zeros(2 * n + m, dtype=float)
    for i in range(m):
        c[i] = c_old[i]
    for i in range(n + m, 2 * n + m):
        c[i] = M

    Cb = np.array([M for _ in range(n)], dtype=float)
    Bp = np.array([i for i in range(m + n, m + 2 * n)], dtype=float) 

    while True:
        F = sum(Cb * b)

        simplex_matrix = np.zeros((n, 4 + m + 2 * n), dtype=float)
        simplex_matrix[:, 0:1] = Bp.reshape(n, 1)
        simplex_matrix[:, 1:2] = Cb.reshape(n, 1)
        simplex_matrix[:, 2:3] = b.reshape(n, 1)
        simplex_matrix[:, 3:-1] = table

        deltas = np.array([c[i] - sum(Cb[j] * table[j, i] for j in range(n)) for i in range(n + m)], dtype=float)
        
        print(f'Промежуточный F_min = {np.round(F, 2)}')

        k = np.argmin(deltas)

        fractions = np.array([b[i] / table[i, k] for i in range(n)], dtype=float)
        fractions = np.array([fractions[i] if fractions[i] > 0 else np.inf for i in range(n)], dtype=float)

        simplex_matrix[:, 2 * n + m + 3: 2 * n + m + 4] = fractions.reshape(n, 1)

        print(f'Δ = {np.round(deltas, 2)}')

        flag = True
        for i in range(n + m):
            if deltas[i] < 0:
                flag = False
                break
        
        if flag:
            return np.round(F, 2), np.round(get_optimal_values(Bp, b, n, m), 2)
        
        print(f'simpex table: \n{pd.DataFrame(np.round(simplex_matrix, 2))}')

        r = np.argmin(fractions)

        table_old = table.copy()

        for i in range(2 * n + m):
            for j in range(n):
                if j == r:
                    table[j, i] = table_old[j, i] / table_old[r, k]
                else:
                    table[j, i] = table_old[j, i] - (table_old[r, i] * table_old[j, k] / table_old[r, k])

        Bp[r] = k
        Cb[r] = c[k]

        b_old = b.copy()
        for i in range(n + m):
            for j in range(n):
                if j == r:
                    b[j] = b_old[j] / table_old[r, k]
                else:
                    b[j] = b_old[j] - (b_old[r] * table_old[j, k] / table_old[r, k])

        print('-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-')

### Тестовые данные

#### Данные с задачи

In [137]:
M = 10 ** 3

c = np.array([560, 170, 300], dtype=float)

A = np.array([
    [2, 1, 2],
    [4, 1, 1]
], dtype=float)

b = np.array([4, 5], dtype=float)

F_min, result = dual_simplex(A, b, c, M=M)
print(f'F_min = {F_min}')
print(f'Оптимальное значение переменных:\n{result}')

Промежуточный F_min = 9000.0
Δ = [-5440. -1830. -2700.  1000.  1000.]
simpex table: 
    0       1    2    3    4    5    6    7    8    9     10
0  5.0  1000.0  4.0  2.0  1.0  2.0 -1.0  0.0  1.0  0.0  2.00
1  6.0  1000.0  5.0  4.0  1.0  1.0  0.0 -1.0  0.0  1.0  1.25
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
Промежуточный F_min = 2200.0
Δ = [    0.  -470. -1340.  1000.  -360.]
simpex table: 
    0       1     2    3     4     5    6     7    8     9    10
0  5.0  1000.0  1.50  0.0  0.50  1.50 -1.0  0.50  1.0 -0.50  1.0
1  0.0   560.0  1.25  1.0  0.25  0.25  0.0 -0.25  0.0  0.25  5.0
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
Промежуточный F_min = 860.0
Δ = [  0.   -23.33   0.   106.67  86.67]
simpex table: 
    0      1    2    3     4    5     6     7     8     9    10
0  2.0  300.0  1.0  0.0  0.33  1.0 -0.67  0.33  0.67 -0.33  3.0
1  0.0  560.0  1.0  1.0  0.17  0.0  0.17 -0.33 -0.17  0.33  6.0
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=

C:\Users\artem\AppData\Local\Temp\ipykernel_19400\666884200.py:60: RuntimeWarning: divide by zero encountered in scalar divide
  fractions = np.array([b[i] / table[i, k] for i in range(n)], dtype=float)


#### Случайные данные

In [138]:
c = np.array([2 + round(2 * random(), 1) for x in range(6)], dtype=float)  
print(F'{c=}\n')

A = np.array([[2 + round(15 * random(), 1) for x in range(6)] for y in range(8)], dtype=float)
print(F'A={pd.DataFrame(A)}\n')

b = np.array([50 + round(50 * random(), 0) for y in range(8)], dtype=float)
print(F'{b=}\n')

c=array([2.6, 3.6, 2.7, 2.9, 3.7, 2.3])

A=      0     1     2     3     4     5
0  14.3  13.2   4.1   6.8   6.4  10.3
1  13.3  11.1  12.3  10.9   9.0   4.8
2  15.5   5.1   7.0   4.1   5.0  14.8
3   5.8   9.1   3.8   4.1   4.1  10.6
4  10.0   6.8   9.5  12.4  13.9   6.6
5  14.2  12.2  16.3  15.0   9.5   9.4
6   2.6   7.5  15.2   9.2  10.3   5.9
7  15.2  15.4   4.6   6.4   5.3  10.0

b=array([55., 50., 54., 67., 66., 76., 67., 92.])



In [139]:
F_min, result = dual_simplex(A, b, c, M=M)
print(f'F_min = {F_min}')
print(f'Оптимальное значение переменных:\n{result}')
np.sum(result[:len(c)] * c)

Промежуточный F_min = 527000.0
Δ = [-90897.4 -80396.4 -72797.3 -68897.1 -63496.3 -72397.7   1000.    1000.
   1000.    1000.    1000.    1000.    1000.    1000. ]
simpex table: 
     0       1     2     3     4     5     6     7     8    9   ...   16   17  \
0  14.0  1000.0  55.0  14.3  13.2   4.1   6.8   6.4  10.3 -1.0  ...  0.0  1.0   
1  15.0  1000.0  50.0  13.3  11.1  12.3  10.9   9.0   4.8  0.0  ...  0.0  0.0   
2  16.0  1000.0  54.0  15.5   5.1   7.0   4.1   5.0  14.8  0.0  ...  0.0  0.0   
3  17.0  1000.0  67.0   5.8   9.1   3.8   4.1   4.1  10.6  0.0  ...  0.0  0.0   
4  18.0  1000.0  66.0  10.0   6.8   9.5  12.4  13.9   6.6  0.0  ...  0.0  0.0   
5  19.0  1000.0  76.0  14.2  12.2  16.3  15.0   9.5   9.4  0.0  ...  0.0  0.0   
6  20.0  1000.0  67.0   2.6   7.5  15.2   9.2  10.3   5.9  0.0  ...  0.0  0.0   
7  21.0  1000.0  92.0  15.2  15.4   4.6   6.4   5.3  10.0  0.0  ... -1.0  0.0   

    18   19   20   21   22   23   24     25  
0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   3.85  


C:\Users\artem\AppData\Local\Temp\ipykernel_19400\666884200.py:60: RuntimeWarning: divide by zero encountered in scalar divide
  fractions = np.array([b[i] / table[i, k] for i in range(n)], dtype=float)
C:\Users\artem\AppData\Local\Temp\ipykernel_19400\666884200.py:60: RuntimeWarning: divide by zero encountered in scalar divide
  fractions = np.array([b[i] / table[i, k] for i in range(n)], dtype=float)


22.68